# The goal of these exercises is to get familiar with pyspark APIs

# Cheat Sheet
**Transformation operations:**
- map: Takes a function as input and applies it to each element in the source RDD to create a new RDD
- flatMap: Takes an input function, which returns a sequence for each input element passed to it returns a new RDD formed by flattening this collection of sequence
- filter: Takes a Boolean function as input and applies it to each element in the source RDD to create a new RDD by selecting only those elements for which the input Boolean function returned true
- distinct: The distinct method of an RDD returns a new RDD containing the distinct elements in the source RDD.
- zip:takes an RDD as input and returns an RDD of pairs, where the first element in a pair is from the source RDD and second element is from the input RDD. Both the source RDD and the input RDD must have the same length.
- groupBy: Groups the elements of an RDD according to a user specified criteria. In each returned pair, the first item is a key and the second item is a collection of the elements mapped to that key by the input function to the groupBy method.
- sortBy: returns an RDD with sorted elements from the source RDD. It takes two input parameters. The first input is a function that generates a key for each element in the source RDD. The second argument allows you to specify ascending or descending order for sort.
- sample: Returns a sampled subset of the source RDD. It takes three input parameters. The first parameter specifies the replacement strategy. The second parameter specifies the ratio of the sample size to source RDD size.
- union: Return the union of this RDD and another one.
- intersection: Return the intersection of this RDD and another one. The output will not contain any duplicate elements, even if the input RDDs did.

**Action operations:**
- collect: Returns the elements in the source RDD as an array. **It can crash the driver program if called on a very large RDD.**
- count: The count method returns a count of the elements in the source RDD.
- countByValue: The countByValue method returns a count of each unique element in the source RDD
- first: The first element in the source RDD.
- max: Returns the largest element in an RDD. Similar idea for min
- stdev: Compute the standard deviation of this RDD’s elements.
- take: takes an integer N as input and returns an array containing the first N element in the source RDD.
- takeOrdered: takes an integer N as input and returns an array containing the N smallest elements in the source RDD.
- top: takes an integer N as input and returns an array containing the N largest elements in the source RDD.
- reduce: aggregates the elements of the source RDD using an associative and commutative binary operator provided to it.

In [1]:
#start the SparkContext
import findspark
findspark.init()
from pyspark import SparkContext 
sc = SparkContext()

# 1. Basic stuff

In [2]:
RDD_text = sc.textFile('Numbers.csv') # reads as text
RDD = RDD_text.map(lambda x: int(x))
# How many numbers are there
print("Count: " + str(RDD.count()))
# Max value
print("Max: " + str(RDD.max()))
# Min value
print("Min: " + str(RDD.min()))
# mean
print("Mean: " + str(RDD.mean()))
# Standard deviation
print("Std. dev. : " + str(RDD.stdev()))

Count: 1000
Max: 100
Min: 1
Mean: 50.812000000000026
Std. dev. : 28.928094579491404


In [3]:
# count of even numbers
print("Count Even: " + str(RDD.filter(lambda x: x%2==0 ).count()))
# count of numbers greater than 80
print("Count > 80: " + str(RDD.filter(lambda x: x>= 80 ).count()))
# sum of odd numbers
print("Sum Odd: " + str(RDD.filter(lambda x: x%2!=0 ).reduce(lambda x,y: x+y))) # or sum()
# number of unique elements in the RDD
print("Count Unique: " + str(RDD.distinct().count()))
# Summation of (x^2 + 5)
print("Summation: " + str(RDD.map(lambda x: x**2+5).reduce(lambda x,y: x+y)))

Count Even: 514
Count > 80: 216
Sum Odd: 25260
Count Unique: 100
Summation: 3423694


# 2. Working with Text

In [4]:
RDD_words = sc.textFile('sonnetWords.txt')
RDD_words = RDD_words.filter(lambda x: x!='')
# How many words are there
print("Count Words: " + str(RDD_words.count()))
# How many unique words
print("Count Unique Words: " + str(RDD_words.distinct().count()))
# How many words having at least 4 characters
print("Count Words > 4 characters: " + str(RDD_words.filter(lambda x: x!='').filter(lambda x: len(x)>4).count()))
# Average number of characters per word
print("Average characters per word: " + str(RDD_words.map(lambda x: len(x)).sum()/RDD_words.count()))
# Number of unique words case-insensitive
print("unique words case-insensitive: " + str(RDD_words.map(lambda x: x.upper()).distinct().count()))
# Convert the words to UPPERCASE and show few samples
print("Upper-case words: " + str(RDD_words.map(lambda x: x.upper()).take(5)))
# print the longest word (having highest number of characters)
print(RDD_words.reduce(lambda w1, w2: w1 if(len(w1)>len(w2)) else w2))

Count Words: 15729
Count Unique Words: 5174
Count Words > 4 characters: 6973
Average characters per word: 4.896751223854028
unique words case-insensitive: 5054
Upper-case words: ['FROM', 'FAIREST', 'CREATURES', 'WE', 'DESIRE']
folly--doctor-like--controlling


In [5]:
def shorterOfTwo (line1, line2):
    if (len(line1.split()) < len(line2.split())): 
        return line1
    else:
        return line2

In [6]:
%%time
RDD_texts = sc.textFile('Moby-Dick.txt')
RDD_texts = RDD_texts.filter(lambda line: line.strip())
print(RDD_texts.take(3))
# How many sentences are there
print("Count Sentences: " + str(RDD_texts.count()))
# Show a sample sentence containing word 'delights'
print("Sample Sentences containing 'delights': " + str(RDD_texts.filter(lambda x: 'delights' in x).take(3)))
# Average number of words per sentence
print("Average number of words per sentence: " + str(RDD_texts.map(lambda line: line.split(" ")).map(lambda x: len(x)).mean()))
# replace all 'crazy' with 'genius' and show 3 examples
print(RDD_texts.map(lambda line: line.replace('crazy', 'genius')).filter(lambda line: 'genius' in line).take(3))
# shortest sentence (in terms of number of words) in the text
print("Sortest sentence: " + RDD_texts.reduce(shorterOfTwo))

['CHAPTER 1. Loomings.', 'Call me Ishmael. Some years ago--never mind how long precisely--having', 'little or no money in my purse, and nothing particular to interest me on']
Count Sentences: 18169
Sample Sentences containing 'delights': ['up towards the warm and pleasant sun, and all the delights of air and', 'all Martial Commanders whom the world invariably delights to honour. And']
Average number of words per sentence: 11.48478177114866
['some time or other genius to go to sea? Why upon your first voyage as a', '"There," said the landlord, placing the candle on a genius old sea chest', "'Twas a foolish, ignorant whim of his genius, widowed mother, who died"]
Sortest sentence: orphan.
Wall time: 11.5 s
